In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from dask.distributed import Client
from distributed.deploy.local import LocalCluster
from pycalphad import Database, equilibrium, variables as v
from pduq.invariant_calc import invariant_samples
from pduq.dbf_calc import eq_calc_samples
from pduq.uq_plot import plot_contour
sns.set(color_codes=True)
import warnings
warnings.filterwarnings("ignore")

In [ ]:
c = LocalCluster(n_workers=8, threads_per_worker=1)
client = Client(c)
print(client)

In [ ]:
dbf = Database('ag-cu_dft.tdb')
params = np.load('trace.npy')[:, -1, :]

In [ ]:
# equlibrium conditions including the pressure (Pa),
# temperature evaluation points (K), and molar
# composition MG evaluation points
conds = {v.P: 101325, v.T: (300, 1600, 10), v.X('CU'): (0, 1, 0.01)}

# perform the equilibrium calculation
eq = eq_calc_samples(dbf, conds, params, client=client)

相边界不确定性

In [ ]:
def plot_binary(eq, comp, alpha=None, cdict=None):

    """
    Plot a binary phase diagram. This purposefully has a
    minimal number of options so that the returned figure
    can be customized easily.

    Parameters
    ----------
    eq : xarray object
        Structured equilibirum calculation
    comp : str
        Label for species to plot on the x-axis,
        e.g. MG for magnesium
    alpha : float, optional
        Number between 0 and 1 for the line transparency
    cdict : dict, optional
        Dictionary with phase names and corresponding
        colors

    Returns
    -------
    compL : numpy array
        1D array with typ values for all parameter sets where
        only the phases in phaseregL are in equilibrium

    Examples
    --------
    >>> import pickle
    >>> import pduq.uq_plot as uq
    >>> with open('single.pkl', 'rb') as buff:
    >>>     eq = pickle.load(buff)
    >>> comp = 'MG'
    >>> # plot a binary phase diagram for a set of
    >>> # equilibrium calculations, and comp as the
    >>> # molar fraction on the x-axis
    >>> uq.plot_binary(eq, comp)
    """

    Tvec = eq.get('T').values  # all temperature values
    Xvec = eq.get('X_' + comp).values  # all molar composition values

    # phaseL: list of unique phases
    phaseL = list(np.unique(eq.get('Phase').values))
    if '' in phaseL:
        phaseL.remove('')
    nph = len(phaseL)  # number of unique phases

    Xph = np.zeros((Tvec.size, Xvec.size, nph))

    for ii in range(nph):
        tmp = eq.X.where(eq.Phase == phaseL[ii])
        tmp = tmp.sel(component=comp)

        nans = np.squeeze(tmp.values)
        nans = np.isnan(nans[..., 0])*np.isnan(nans[..., 1])
        Xph_ = np.squeeze(tmp.sum(dim='vertex').values)
        Xph_[nans] = np.nan
        Xph[..., ii] = Xph_

    XphD = {}

    for ii in range(Tvec.size):
        T = str(np.int32(Tvec[ii]))
        xl = 'X_' + T
        pl = 'Ph_' + T
        XphD[xl], XphD[pl] = [], []
        for jj in range(nph):
            # find values not equal to the composition
            neq2comp = np.isclose(Xph[ii, :, jj], Xvec, atol=1e-4)
            neq2comp = np.invert(neq2comp)
            vals = Xph[ii, neq2comp, jj]

            # remove nans
            vals = vals[np.invert(np.isnan(vals))]

            # get unique values
            vals_ = np.round(vals, 5)
            loc, indx = np.unique(vals_, return_index=True)
            vals = vals[indx]

            XphD[xl] += list(vals)
            XphD[pl] += len(vals)*[phaseL[jj]]

        arg = np.argsort(np.array(XphD[xl]))
        XphD[xl] = np.array(XphD[xl])[arg]
        XphD[pl] = np.array(XphD[pl])[arg]

    colorL = sns.color_palette("cubehelix", nph)
    if alpha is None:
        alpha = 0.9

    for ii in range(Tvec.size):
        T = str(np.int32(Tvec[ii]))
        xl = 'X_' + T
        pl = 'Ph_' + T
        vals = XphD[xl]
        phs = XphD[pl]
        for jj in range(nph):
            if phaseL[jj] not in list(phs):
                continue
            vals_ = vals[phs == phaseL[jj]]

            if cdict is not None:
                color = cdict[phaseL[jj]]
            else:
                color = colorL[jj]

            plt.plot(vals_, [Tvec[ii]]*len(vals_),
                     marker='o', markersize=2, alpha=alpha,
                     color=color, linestyle='', mew=0.0)

In [ ]:
def plot_superimposed(
        eq, comp, nsp=None, alpha=None,
        phase_label_dict=None,
        xlims=None, cdict=None, figsize=None):
    phaseL = list(np.unique(eq.get('Phase').values))
    if '' in phaseL:
        phaseL.remove('')
    nph = len(phaseL)

    if nsp is None:
        nsp = len(eq.sample)

    fig, ax = plt.subplots(figsize=figsize)

    # plot the phase boundaries for each parameter set
    for ii in range(nsp):
        plot_binary(
            eq.sel(sample=ii), comp=comp, alpha=alpha, cdict=cdict)
        print('diagram', ii, 'plotted')

    # plot the legend
    nph = len(phaseL)
    colorL = sns.color_palette("cubehelix", nph)
    Tvec = eq.get('T').values
    for ii in range(nph):
        phase = phaseL[ii]
        if cdict is not None:
            color = cdict[phaseL[ii]]
        else:
            color = colorL[ii]
        if phase_label_dict is not None:
            label = phase_label_dict[phase]
        else:
            label = phase
        plt.plot(1.5, Tvec[0], color=color, linestyle='',
                 marker='.', label=label)

    if xlims is None:
        Xvec = eq.get('X_' + comp).values
        Xrng = Xvec.max() - Xvec.min()
        xlims = [Xvec.min() - 0.005*Xrng, Xvec.max() + 0.005*Xrng]

    plt.xlim(xlims)
    plt.ylim([1050, 1400])
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    plt.xlabel(r'$\mathrm{x_{%s}}$' % comp, fontsize='large')
    plt.ylabel('T (K)', fontsize='large')
    plt.legend()
    plt.tight_layout()

In [ ]:
phaseL = list(np.unique(eq.get('Phase').values))
if '' in phaseL: phaseL.remove('')
nph = len(phaseL)
colorL = sns.color_palette("cubehelix", nph+2)
cdict = {}
for ii in range(nph):
    cdict[phaseL[ii]] = colorL[ii]

# we can also specify a dictionary to have pretty
# labels (even LaTeX) in our plots
phase_label_dict = {
    'LIQUID':'liquid', 'FCC_A1':'FCC'}

# plot the superimposed phase diagrams
import matplotlib.pyplot as plt
#plt.ylim([1000, 1500])
# plot the superimposed phase diagrams
#Tvec.min()=1000
plot_superimposed(eq, 'CU', alpha=0.2,
                  xlims=[0, 1], cdict=cdict,
                  phase_label_dict=phase_label_dict,
                  figsize=(5, 4))

plt.savefig('newPhase_Diagram_Uncertainty.png',dpi=600)
plt.show()

In [ ]:
from pduq.uq_plot import plot_phasereg_prob
coordD = 'X_CU'
plot_phasereg_prob(
    eq, ['LIQUID'], coordplt=[coordD, 'T'], figsize=(5, 3))
plt.savefig('newPhase_Diagram_Uncertainty2.png',dpi=600)
#plt.show()

In [ ]:
coordD = 'X_CU'
plot_phasereg_prob(
    eq, ['FCC_A1'], coordplt=[coordD, 'T'], figsize=(5, 3))
plt.savefig('newPhase_Diagram_Uncertainty3.png',dpi=600)
plt.show()

相分数

In [ ]:
def plot_phasefracline(eq, coordD, xlabel=None,
                       phase_label_dict=None, cdict=None, figsize=None):

    """
    Plot the phase fraction with uncertainty versus composition,
    temperature or pressure.

    Parameters
    ----------
    eq : xarray object
        Structured equilibirum calculation containing a 'sample'
        dimension correspoinding to different parameter sets
    coordD : dict
        Dictionary defining constraints on the coordinates in
        eq for plotting the phase fraction with varying X, T or P
        For example, we might pick a fixed X and let T vary
        as follows: coordD = {'X_MG':0.1}
    xlabel : str, optional
        Label for the x-axis
    phase_label_dict : dict, optional
        Dictionary with keys given by phase names and corresponding
        strings to use in plotting (e.g. to enable LaTeX labels)
    xlims : list or tuple of float, optional
        List or tuple with two floats corresponding to the
        minimum and maximum molar composition of comp
    cdict : dict, optional
        Dictionary with phase names and corresponding
        colors
    figsize : tuple or list of int or float, optional
        Plot dimensions in inches

    Returns
    -------

    Examples
    --------
    >>> import pickle
    >>> import pduq.uq_plot as uq
    >>> with open('full.pkl', 'rb') as buff:
    >>>     eq = pickle.load(buff)
    >>> # if for 'full.pkl' X_MG and T have 100 intervals each
    >>> # we can fix T and plot the phase fraction versus
    >>> # X_MG with uncertainty
    >>> coordD = {'T':1000}
    >>> # plot phase fraction versus X_MG
    >>> uq.plot_phasefracline(eq, coordD, xlabel='X_MG')
    """
    phaseL = list(np.unique(eq.get('Phase').values))
    if '' in phaseL:
        phaseL.remove('')
    nph = len(phaseL)

    eq = eq.sel(coordD)

    # the X, T or P dimension with the most values
    # is then plotted on the x-axis
    rmlist = ['N', 'internal_dof', 'sample', 'vertex']
    max_sz = 0
    for dim in list(eq.dims):
        if dim not in rmlist:
            dim_sz = eq.dims[dim]
            if dim_sz > max_sz:
                max_sz = dim_sz
                dim_max = dim
    xvec = eq.get(dim_max).values

    CI = 95
    colorL = sns.color_palette("cubehelix", nph)

    plt.figure(figsize=figsize)

    for ii in range(nph):
        phase = phaseL[ii]
        val = eq.NP.where(eq.Phase == phase)
        val = val.sum(dim='vertex').values.squeeze()

        low, mid, high = np.percentile(
            val, [0.5*(100-CI), 50, 100-0.5*(100-CI)], axis=0)

        if phase_label_dict is not None:
            label = phase_label_dict[phase]
        else:
            label = phase

        if cdict is not None:
            color = cdict[phaseL[ii]]
        else:
            color = colorL[ii]

        plt.plot(xvec, mid, linestyle='-', color=color, label=label)
        plt.fill_between(
            np.atleast_1d(xvec), low, high, alpha=0.3, facecolor=color)

    plt.xlim([1020, 1100])
    plt.ylim([-0.01, 1.01])
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    plt.xlabel(xlabel, fontsize='large')
    plt.ylabel('phase fraction', fontsize='large')
    plt.legend()
    plt.tight_layout()

In [ ]:
# define a dictionary fixing the composition. We need to pick
# a value for X_MG that is represented in the eq object
coordD = {'X_CU':0.4}

# plot the phase fraction versus composition
# cdict and phase_label_dict are the same as in the previous example
plot_phasefracline(eq, coordD, xlabel='T (K)', cdict=cdict,
                      phase_label_dict=phase_label_dict, figsize=(5, 3))
plt.savefig('newphase_fraction0-4.png',dpi=600)
plt.show()

热力学性质不确定性

In [ ]:
from espei.core_utils import get_data#(0.8.3版本需要使用该函数)
#from espei.core_utils import filter_configurations #(大于0.8.3版本需要使用该函数)
from pycalphad import calculate
from espei.utils import database_symbols_to_fit
from collections import OrderedDict
import xarray as xr

In [ ]:
def plot_property(dbf, comps, phaseL, params, T, prop,
                  config=None, datasets=None, xlim=None,
                  xlabel=None, ylabel=None, yscale=None,
                  phase_label_dict=None,
                  unit='kJ/mol.', cdict=None, figsize=None):

    symbols_to_fit = database_symbols_to_fit(dbf)

    CI = 95
    nph = len(phaseL)
    colorL = sns.color_palette("cubehelix", nph)
    markerL = 10*['o', 'D', '^', 'x', 'h', 's',
                  'v', '*', 'P', 'p', '>', 'd', '<']

    plt.figure(figsize=figsize)

    # compute uncertainty in property for each phase in list
    for ii in range(nph):
        phase = phaseL[ii]
        print('starting', prop, 'evaluations for the', phase, 'phase')

        # for each parameter sample calculate the property
        # for each possible site occupancy ratios
        compL = []
        for index in range(params.shape[0]):
            param_dict = {param_name: param for param_name, param
                          in zip(symbols_to_fit, params[index, :])}
            parameters = OrderedDict(sorted(param_dict.items(), key=str))
            comp = calculate(
                dbf, comps, phase,
                P=101325, T=T, output=prop, parameters=parameters)
            compL += [comp]

        # concatenate the calculate results in an xarray along
        # an axis named 'sample'
        compC = xr.concat(compL, 'sample')
        compC.coords['sample'] = np.arange(params.shape[0])

        # The composition vector is the same for all samples
        if hasattr(T, "__len__"):
            Xvals = T
        else:
            Xvals = comp.X.sel(component=comps[0]).values.squeeze()
        Pvals = compC[prop].where(compC.Phase == phase).values.squeeze()

        if np.array(Xvals).size == 1:
            print('phase is a line compound')
            Xvals_ = np.array([Xvals-0.002, Xvals+0.002])
            Pvals_ = np.vstack([Pvals, Pvals]).T
        else:
            # find the lower hull of the property by finding
            # the configuration with the lowest value within
            # each interval. In each interval record the composition
            # and property
            indxL = np.array([])
            # Xbnds = np.arange(0, 1.01, 0.01)
            Xbnds = np.linspace(Xvals.min(), Xvals.max(), 100)
            for lb, ub in zip(Xbnds[:-1], Xbnds[1:]):
                # print('lb: ', lb, ', ub: ', ub)
                boolA = (lb <= Xvals)*(Xvals < ub)
                if boolA.sum() == 0:
                    continue
                indxA = np.arange(boolA.size)[boolA]
                P_ = Pvals[0, boolA]
                indxL = np.append(indxL, indxA[P_.argmin()])
                # indxL = np.append(indxL, indxA[P_.argmax()])
            indxL = indxL.astype('int32')

            if indxL.size == 1:
                print('only one point found')
                Xvals_ = Xvals[np.asscalar(indxL)]
                Pvals_ = Pvals[:, np.asscalar(indxL)]
            else:
                Xvals_ = Xvals[indxL]
                Pvals_ = Pvals[:, indxL]

        # Xvals_ = Xvals
        # Pvals_ = Pvals
        # for ii in range(params.shape[0]):
        #     plt.plot(Xvals_, Pvals_[ii, :], 'k-', linewidth=0.5, alpha=0.1)
        # plt.show()

        if yscale is not None:
            Pvals_ *= yscale

        low, mid, high = np.percentile(
            Pvals_, [0.5*(100-CI), 50, 100-0.5*(100-CI)], axis=0)

        if cdict is not None:
            color = cdict[phase]
        else:
            color = colorL[ii]

        if phase_label_dict is not None:
            label = phase_label_dict[phase]
        else:
            label = phase

        plt.plot(Xvals_, mid, linestyle='-', color=color, label=label)
        plt.fill_between(
            np.atleast_1d(Xvals_), low, high, alpha=0.3, facecolor=color)

        # collect and plot experimental data
        if config is not None and datasets is not None:
            symmetry = None
            data = get_data(
                comps, phase, config, symmetry, datasets, prop)#0.8.3版本使用
            '''filtered_data = filter_temperatures(datasets)
            data = filter_configurations(filtered_data, config, symmetry)'''#大于0.8.3版本使用
            print(data)
            for data_s, marker in zip(data, markerL):
                occupancies = data_s['solver']['sublattice_occupancies']
                # at the moment this needs to be changed manually
                X_vec = [row[0][0] for row in occupancies]
                values = np.squeeze(data_s['values'])

                if yscale is not None:
                    values *= yscale

                plt.plot(
                    X_vec, values, linestyle='', marker=marker,
                    markerfacecolor='none', markeredgecolor=color,
                    markersize=6, alpha=0.9, label=data_s['reference'])

    if xlim is None:
        plt.xlim([Xvals_.min(), Xvals_.max()])
    else:
        plt.xlim(xlim)

    if xlabel is not None:
        plt.xlabel(xlabel)
    else:
        plt.xlabel(r'$X_{%s}$' % comps[0])

    if ylabel is not None:
        plt.ylabel(ylabel)
    else:
        plt.ylabel(prop + ' (' + unit + ')')

    plt.legend()
    plt.tight_layout()

In [ ]:
#from pduq.uq_plot import plot_property
comps = ['CU', 'AG', 'VA'] # species to consider
T = 1100  # temperature in Kelvin
prop = 'GM'  # property of interest (molar Gibbs energy)
ylabel = 'Molar Gibbs energy\n(kJ/mol.K)'  # y-axis label
yscale = 1e-3  # we want to scale the Gibbs energy to give kJ/mol.

# we can now plot the property. Note that phaseL, phase_label_dict,
# and cdict are defined in the phase diagram prediction example
plot_property(dbf, comps, phaseL, params, T, prop,
                 phase_label_dict=phase_label_dict,
                 ylabel=ylabel, yscale=yscale, cdict=cdict,
                 xlim=[-0.005, 1.005], figsize=(5, 3))
plt.savefig('newphase_Molar_Gibbs_Energy.png',dpi=600)
plt.show()

In [ ]:
T = 1056  # temperature in Kelvin
prop = 'HM_MIX'  # property of interest (molar enthalpy of mixing
ylabel = 'Molar enthalpy of mixing\n(kJ/mol.)'  # y-axis label
yscale = 1e-3  # we want to scale the Gibbs energy to give kJ/mol.

# we can now plot the property. Note that phaseL, phase_label_dict,
# and cdict are defined in the phase diagram prediction example
hm_mix=plot_property(dbf, comps, phaseL, params, T, prop,
                 phase_label_dict=phase_label_dict,
                 ylabel=ylabel, yscale=yscale, cdict=cdict,
                 xlim=[-0.005, 1.005], figsize=(5, 3))
plt.savefig('newphase_Molar_Enthalpy_of_Mixing.png',dpi=600)

相变点概率分布

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from dask.distributed import Client
from distributed.deploy.local import LocalCluster
from pycalphad import Database, equilibrium, variables as v
from pduq.invariant_calc import invariant_samples
from pduq.uq_plot import plot_contour
sns.set(color_codes=True)
c = LocalCluster(n_workers=8, threads_per_worker=1)
client = Client(c)
print(client)

In [ ]:
X = 0.4  # X guess for invariant
P = 101324  # pressure
Tl = 300  # lower temperature bound for invariant
Tu = 1600  # upper temperature bound for invariant
comp = 'CU' # species to reference for composition
dbf = Database('ag-cu_dft.tdb')
params = np.load('trace.npy')[:, -1, :]

In [ ]:
import logging
import pickle
import sympy
import numpy as np
import xarray as xr
from collections import OrderedDict
from espei.utils import database_symbols_to_fit
from itertools import chain
from pycalphad import equilibrium, variables as v
from pycalphad.codegen.callables import build_callables
from pycalphad.core.utils import instantiate_models
from time import time
import logging
import numpy as np
#from .dbf_calc import eq_calc_  # , get_eq_callables_
from espei.utils import database_symbols_to_fit
from pycalphad import variables as v

comps = list(dbf.elements)
phases = list(dbf.phases.keys())
neq = params.shape[0]
bndv = np.zeros((neq, 2))

neq = params.shape[0]  # calculate invariants for neq parameter sets

symbols_to_fit = database_symbols_to_fit(dbf)

# eq_callables = get_eq_callables_(dbf, comps, phases, symbols_to_fit)
eq_callables = None  # eq_callables is disabled for current pycalcphad
kwargs = {'dbf': dbf, 'comps': comps, 'phases': phases,
              'X': X, 'P': P, 'Tl': Tl, 'Tu': Tu, 'comp': comp,
              'params': params, 'symbols_to_fit': symbols_to_fit,
              'eq_callables': eq_callables}

In [ ]:
def eq_calc_(dbf, comps, phases, conds,
             paramA, symbols_to_fit,
             eq_callables=None):

    param_dict = {param_name: param for param_name, param
                  in zip(symbols_to_fit, paramA)}

    parameters = OrderedDict(sorted(param_dict.items(), key=str))

    eq_result = equilibrium(dbf, comps, phases, conds,
                            parameters=parameters, callables=eq_callables)

    return eq_result

In [ ]:
def invariant_(index, dbf, params, comps, phases, X, P, Tl, Tu, comp,
               symbols_to_fit, eq_callables):

    kwargs = {'dbf': dbf, 'comps': comps, 'phases': phases,
              'paramA': params[index, :], 'symbols_to_fit': symbols_to_fit,
              'eq_callables': eq_callables}

    def mini(T):
        """
        perform the equilibrium calculation at a specified temperature
        and return the list of unique phases in equilibrium along with the
        equlibrium data object
        """
        conds = {v.P: P, v.T: T, v.X(comp): X}
        eq = eq_calc_(conds=conds, **kwargs)
        PhT = list(np.unique(eq.Phase))
        if '' in PhT:
            PhT.remove('')
        return eq, PhT

    # perform equilibrium calculations at the lower bound, middle, and
    # upper bound temperatures
    Tm = 0.5*(Tl+Tu)  # middle temperature
    eql, PhTl = mini(Tl)
    eqm, PhTm = mini(Tm)
    equ, PhTu = mini(Tu)

    # now we use the bisection method to find the invariant temperature
    # and composition

    # identify the number of calculations so that the error in the
    # temperature is less than errlim
    errlim = 0.01
    niter = np.log(errlim/(Tu-Tl))/np.log(0.5) - 1
    niter = np.int16(np.ceil(niter))

    for ii in range(niter):
        # if the phases in equilibrium at the middle temperature are
        # different than at the lower temperature, then the invariant
        # will be between the lower and middle temperature. We can then
        # set the upper temperature to the previous middle temperature.
        # Otherwise, the invariant will be between the middle and upper
        # temperature.
        if str(PhTm) != str(PhTl):
            Tu = Tm
            equ = eqm
            PhTu = PhTm
        else:
            Tl = Tm
            eql = eqm
            PhTl = PhTm
        Tm = 0.5*(Tl + Tu)  # get the new middle temperature
        eqm, PhTm = mini(Tm)  # calculate the Tm equilibrium
        # print(Tm, ' ', PhTm)

    def getbnd(eq, PhT):
        """
        get the molar compositions of the phases in PhT
        """
        bnd = []
        for phase in PhT:
            tmp = eq.X.where(eq.Phase == phase)
            tmp = tmp.sel(component=comp).sum(dim='vertex')
            bnd.append(np.squeeze(tmp.values))
        return np.array(bnd)

    # PhTA are the phases at Tl and Tu
    PhTA = np.concatenate([PhTl, PhTu])
    # bndA are the molar compositions of the phases in PhTl and PhTu
    bndA = np.concatenate([getbnd(eql, PhTl), getbnd(equ, PhTu)])
    # phs are the unique phases in the three phase region
    phs, indx = np.unique(PhTA, return_index=True)
    # bnd are the molar compositions corresponding to phs
    bnd = bndA[indx]

    indx = np.argsort(bnd)
    phs = list(phs[indx])
    bnd = bnd[indx]

    logging.info('Invariant computed for set ' + str(index))
    logging.info('T = ' + str(Tm) + ' +/- ' + str(Tm-Tl) + 'K')
    logging.info(str(phs))
    logging.info(str(bnd))

    # print('Invariant computed for set ', index)
    # print('T=', Tm, '+/-', Tm-Tl, 'K')
    #print(phs)
    # print(bnd)

    return Tm, phs, bnd, index

In [ ]:
invL = []
for ii in range(neq):
    invL.append(invariant_(ii, **kwargs))

In [ ]:
Tv = np.zeros((neq,))
phv = neq*[None]
bndv = np.zeros((neq, 2))
for ii in range(neq):
    Tv[ii] = invL[ii][0]
    phv[ii] = invL[ii][1]
    bndv[ii] = invL[ii][2]

In [ ]:
print('mean invariant composition:', np.mean(bndv[:, 0]))
print('mean invariant temperature:', np.mean(Tv))

In [ ]:
'''import numpy as np
bndvee = np.array(bndv[:, 0])
Tvee = np.array(Tv)
phvee = np.array(phv)
np.save('C:/Users/fsun/phasediagram/agcu/testtrigle/morepara/new0-39comp',bndvee) 
np.save('C:/Users/fsun/phasediagram/agcu/testtrigle/morepara/new0-39T',Tvee) 
np.save('C:/Users/fsun/phasediagram/agcu/testtrigle/morepara/new0-39phase',phv) '''

In [ ]:
from scipy.stats import gaussian_kde
def plot_contour(points, c='k', bw=0.3):

    """
    Plot as set of KDE probability density contours for a set
    of points, typically corresponding to invariant locations

    Parameters
    ----------
    points : numpy array
        an array of compositions and temperatures representing
        invariant points or some other phase diagram feature
    c : color, optional
        color of the density contours
    bw : float, optional
        KDE bandwidth

    Returns
    -------

    Examples
    --------
    >>> import numpy as np
    >>> from pduq.invariant_calc import invariant_samples
    >>> from pduq.uq_plot import plot_contour
    >>> from pycalphad import Database
    >>> # load dbf file and raw parameter set.
    >>> dbf = Database('CU-MG_param_gen.tdb')
    >>> params = np.loadtxt('trace.csv', delimiter=',')
    >>> # find the set of invariant points
    >>> Tv, phv, bndv = invariant_samples(
    >>>     dbf, params, X=.2, P=101325, Tl=600, Tu=1400,
    >>>     comp='MG')
    >>> # define the 'points' array
    >>> points = np.zeros((len(Tv, 2)))
    >>> points[:, 0] = bndv[:, 1]
    >>> points[:, 1] = Tv
    >>> # plot the contour
    >>> plt.figure()
    >>> uq.plot_contour(points)
    """
    kernel = gaussian_kde(points.T)

    buf_mult = 0.5

    xmin = points[:, 0].min()
    xmax = points[:, 0].max()
    buf = buf_mult*(xmax-xmin)
    xvec = np.linspace(xmin-buf, xmax+buf, 100)

    ymin = points[:, 1].min()
    ymax = points[:, 1].max()
    buf = buf_mult*(ymax-ymin)
    yvec = np.linspace(ymin-buf, ymax+buf, 100)

    X, Y = np.meshgrid(xvec, yvec)
    XY = np.vstack([X.ravel(), Y.ravel()])

    kernel = gaussian_kde(points.T, bw_method=bw)
    pdf = kernel(XY).T
    Z = pdf.reshape(X.shape)

    order = pdf.argsort()
    pdf_s = pdf[order]
    F = np.cumsum(pdf_s)
    F /= F[-1]

    Plvls = np.array([1-.9545, 1-.6827])
    boundaries = F.searchsorted(Plvls)
    lvls = pdf_s[boundaries]

    CS = plt.contour(X, Y, Z, lvls, colors=c, alpha=.7, linestyles=[':', '-'])
    labels = ["95% CI", "68% CI"]

    for ii in range(len(labels)):
        CS.collections[ii].set_label(labels[ii])

In [ ]:
points = np.zeros((len(Tv), 2))
points[:, 0] = bndv[:, 0]
points[:, 1] = Tv
c = sns.color_palette("Blues", 2)

plt.figure(figsize=(5, 3))

# plot the raw invariant points
plt.plot(points[:, 0], points[:, 1], 'k.',
         label="invariant\nsamples")

# plot KDE estimated uncertainty intervals
#labels = ["96% CI", "68% CI"]
plot_contour(points, c, 0.9)

plt.xlabel(r'$\mathrm{x_{Cu}}$', fontsize="large")
plt.ylabel('T (K)', fontsize="large")
#my_x_ticks = np.arange(0.36, 0.42, 0.01)
#my_y_ticks = np.arange(1050, 1065, 5)
#plt.xticks(my_x_ticks)
#plt.yticks(my_y_ticks)
plt.tight_layout()
plt.legend()
#plt.show()
plt.savefig('newFigure2.jpg', bbox_inches = "tight",dpi=600)

相稳定性概率分布

In [ ]:
import numpy as np
from dask.distributed import Client
from distributed.deploy.local import LocalCluster
from pycalphad import Database, variables as v
from pduq.dbf_calc import eq_calc_samples
from pduq.uq_plot import get_phase_prob, plot_dist
import warnings
warnings.filterwarnings("ignore")

In [ ]:
c = LocalCluster(n_workers=8, threads_per_worker=1)
client = Client(c)
print(client)

In [ ]:
dbf = Database('ag-cu_dft.tdb')
params = np.load('trace.npy')[:, -500:, :]
print(np.load('trace.npy').shape)
print(np.load('trace.npy')[:, -1, :].shape)
print(np.load('trace.npy')[:, -500:, :].shape)

In [ ]:
params = params.reshape(60000,12)
print(params.shape)

In [ ]:
# Define the equilibrium conditions including pressure
# (Pa), temperature (K), and molar composition Mg
conds = {v.P: 101325, v.T: 1056, v.X('CU'): 0.391}

# perform the equilibrium calculation for all parameter
# sets
eq = eq_calc_samples(dbf, conds, params, client=client)

In [ ]:
phaseregLL = [['FCC_A1', 'LIQUID'], ['LIQUID'],
              ['FCC_A1']]

# calculate the probability of non-zero phase fraction
for phaseregL in phaseregLL:
    prob = get_phase_prob(eq, phaseregL)
    print(phaseregL, ' ', np.squeeze(prob.values))

In [ ]:
import matplotlib.pyplot as plt
#coordD = {'X_CU':0.1}
coordD = {'T':1056, 'X_CU':0.391, 'component':'CU'}
#v.X('MG'): (0, 1, 0.01)
phaseregL = ['FCC_A1','LIQUID']
phase = 'FCC_A1'

# plot the phase fraction
plot_dist(eq, coordD, phaseregL, phase, typ='NP', figsize=(5, 3))
plt.savefig("NP3.png",dpi=600)
# plot the phase composition
plot_dist(eq, coordD, phaseregL, phase, typ='X', figsize=(5, 3))

plt.savefig("X3.png",dpi=600)
# plot molar Gibbs energy of the X-T-P point
plot_dist(eq, coordD, phaseregL, phase, typ='GM', figsize=(5, 3))
plt.savefig("GM3.png",dpi=600)
# chemical potential of the selected component
plot_dist(eq, coordD, phaseregL, phase, typ='MU', figsize=(5, 3))
plt.savefig("MU3.png",dpi=600)